Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Extractive Summarization on CNN/DM Dataset using Transformer Version of BertSum


### Summary

This notebook demonstrates how to fine tune Transformers for extractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

BertSum refers to  [Fine-tune BERT for Extractive Summarization](https://arxiv.org/pdf/1903.10318.pdf) with [published example](https://github.com/nlpyang/BertSum/). And the Transformer version of Bertsum refers to our modification of BertSum and the source code can be accessed at (https://github.com/daden-ms/BertSum/). 

Extractive summarization are usually used in document summarization where each input document consists of mutiple sentences. The preprocessing of the input training data involves assigning label 0 or 1 to the document sentences based on the give summary. The summarization problem is also simplfied to classifying whether a document sentence should be included in the summary. 

The figure below illustrates how BERTSum can be fine tuned for extractive summarization task. [CLS] token is inserted at the beginning of each sentence, so is [SEP] token at the end. Interval segment embedding and positional embedding are added upon the token embedding as the input of the BERT model. The [CLS] token representation is used as sentence embedding and only the [CLS] tokens are used as the input for the summarization model. The summarization layer predicts the probability for each  sentence being included in the summary. Techniques like trigram blocking can be used to improve model accuarcy.   

<img src="https://nlpbp.blob.core.windows.net/images/BertSum.PNG">


### Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Ubuntu Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

Using only 1 NVIDIA Tesla V100 GPUs, 16GB GPU memory configuration,
- for data preprocessing, it takes around 1 minutes to preprocess the data for quick run. Otherwise it takes ~2 hours to finish the data preprocessing. This time estimation assumes that the chosen transformer model is "distilbert-base-uncased" and the sentence selection method is "greedy", which is the default. The preprocessing time can be significantly longer if the sentence selection method is "combination", which can achieve better model performance.

- for model fine tuning, it takes around 10 minutes for quick run. Otherwise, it takes around ~3 hours to finish. This estimation assumes the chosen encoder method is "transformer". The model fine tuning time can be shorter if other encoder method is chosen, which may result in worse model performance. 


In [1]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True
## Set USE_PREPROCSSED_DATA = True to skip the data preprocessing
USE_PREPROCSSED_DATA = False

### Configuration


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
from utils_nlp.eval.evaluate_summarization import get_rouge
from utils_nlp.models.transformers.extractive_summarization import (
    ExtractiveSummarizer,
    ExtSumProcessedData,
    ExtSumProcessor,
)

import pandas as pd
import scrapbook as sb

/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu


### Configuration: choose the transformer model to be used

Several pretrained models have been made available by [Hugging Face](https://github.com/huggingface/transformers). For extractive summarization, the following pretrained models are supported. 

In [5]:
pd.DataFrame({"model_name": ExtractiveSummarizer.list_supported_models()})

,model_name
0,bert-base-uncased
1,distilbert-base-uncased


In [6]:
# Transformer model being used
MODEL_NAME = "distilbert-base-uncased"

Also, we need to install the dependencies for pyrouge.

# dependencies for ROUGE-1.5.5.pl
Run the following commands in your terminal to install XML parsing C library.

1. sudo apt-get update
1. sudo apt-get install expat
1. sudo apt-get install libexpat-dev -y

Run the following commands in your terminal to install other pre-requistes for using pyrouge.
1. sudo cpan install XML::Parser
1. sudo cpan install XML::Parser::PerlSAX
1. sudo cpan install XML::DOM

Download ROUGE-1.5.5 from https://github.com/andersjo/pyrouge/tree/master/tools/ROUGE-1.5.5.
Run the following command in your terminal.
* pyrouge_set_rouge_path $ABSOLUTE_DIRECTORY_TO_ROUGE-1.5.5.pl

### Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples.  You can choose the [Option 1] below preprocess the data or [Option 2] to use the preprocessed version at [BERTSum published example](https://github.com/nlpyang/BertSum/). You don't need to manually download any of these two data sets as the code below will handle downloading. Functions defined specific in [cnndm.py](../../utils_nlp/dataset/cnndm.py) are unique to CNN/DM dataset that's preprocessed by harvardnlp. However, it provides a skeleton of how to preprocessing text into the format that model preprocessor takes: sentence tokenization and work tokenization. 

##### Details of Data Preprocessing

The purpose of preprocessing is to process the input articles to the format that model finetuning needed. Assuming you have (1) all articles and (2) target summaries, each in a file and line-breaker separated, the steps to preprocess the data are:
1. sentence tokenization
2. word tokenization
3. **label** the sentences in the article with 1 meaning the sentence is selected and 0 meaning the sentence is not selected. The algorithms for the sentence selection are "greedy" and "combination" and can be found in [sentence_selection.py](../../utils_nlp/dataset/sentence_selection.py)
3. convert each example to  the desired format for extractive summarization
    - filter the sentences in the example based on the min_src_ntokens argument. If the lefted total sentence number is less than min_nsents, the example is discarded.
    - truncate the sentences in the example if the length is greater than max_src_ntokens
    - truncate the sentences in the example and the labels if the total number of sentences is greater than max_nsents
    - [CLS] and [SEP] are inserted before and after each sentence
    - wordPiece tokenization or Byte Pair Encoding (BPE) subword tokenization
    - truncate the example to 512 tokens
    - convert the tokens into token indices corresponding to the transformer tokenizer's vocabulary.
    - segment ids are generated and added
    - [CLS] token positions are logged
    - [CLS] token labels are truncated if it's greater than 512, which is the maximum input length that can be taken by the transformer model.
    
    
Note that the original BERTSum paper use Stanford CoreNLP for data preprocessing, here we use NLTK for data preprocessing. 

##### [Option 1] Preprocess  data (Please skil this part if you choose to use preprocessed data)
The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.

In [11]:
# the data path used to save the downloaded data file
DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 1000
CHUNK_SIZE=200
if not QUICK_RUN:
    TOP_N = -1
    CHUNK_SIZE = 2000

In [19]:
train_dataset, test_dataset = CNNDMSummarizationDataset(top_n=TOP_N, local_cache_path=DATA_PATH)

I0103 05:29:37.485339 140060135520064 utils.py:173] Opening tar file /tmp/tmpjd6tv6g9/cnndm.tar.gz.
I0103 05:29:37.487093 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/test.txt.src already extracted.
I0103 05:29:37.777695 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/test.txt.tgt.tagged already extracted.
I0103 05:29:37.804513 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/train.txt.src already extracted.
I0103 05:29:45.345131 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/train.txt.tgt.tagged already extracted.
I0103 05:29:45.963999 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/val.txt.src already extracted.
I0103 05:29:46.300785 140060135520064 utils.py:181] /tmp/tmpjd6tv6g9/val.txt.tgt.tagged already extracted.


Preprocess the data and save the data to disk.

In [20]:
processor = ExtSumProcessor(model_name=MODEL_NAME)
ext_sum_train = processor.preprocess(train_dataset, train_dataset.get_target(), oracle_mode="greedy")
ext_sum_test = processor.preprocess(test_dataset, test_dataset.get_target(),oracle_mode="greedy")

I0103 05:29:49.643863 140060135520064 tokenization_utils.py:379] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [21]:
save_path = os.path.join(DATA_PATH, "processed")
train_files = ExtSumProcessedData.save_data(
    ext_sum_train, is_test=False, save_path=save_path, chunk_size=CHUNK_SIZE
)
test_files = ExtSumProcessedData.save_data(
    ext_sum_test, is_test=True, save_path=save_path, chunk_size=CHUNK_SIZE
)

In [22]:
train_files

['/tmp/tmpjd6tv6g9/processed/0_train',
 '/tmp/tmpjd6tv6g9/processed/1_train',
 '/tmp/tmpjd6tv6g9/processed/2_train',
 '/tmp/tmpjd6tv6g9/processed/3_train',
 '/tmp/tmpjd6tv6g9/processed/4_train']

In [23]:
test_files

['/tmp/tmpjd6tv6g9/processed/0_test',
 '/tmp/tmpjd6tv6g9/processed/1_test',
 '/tmp/tmpjd6tv6g9/processed/2_test',
 '/tmp/tmpjd6tv6g9/processed/3_test',
 '/tmp/tmpjd6tv6g9/processed/4_test']

In [24]:
train_dataset, test_dataset = ExtSumProcessedData().splits(root=save_path)

#### Inspect Data

In [25]:
import torch
bert_format_data = torch.load(train_files[0])
print(len(bert_format_data))
bert_format_data[0].keys()

200


dict_keys(['src', 'labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])

In [26]:
bert_format_data[0]['labels']

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

##### [Option 2] Reuse Preprocessed  data from [BERTSUM Repo](https://github.com/nlpyang/BertSum)

In [27]:
# the data path used to downloaded the preprocessed data from BERTSUM Repo.
# if you have downloaded the dataset, change the code to use that path where the dataset is.
PROCESSED_DATA_PATH = TemporaryDirectory().name
#data_path = "./temp_data5/"
#PROCESSED_DATA_PATH = data_path

In [28]:
if USE_PREPROCSSED_DATA:
    CNNDMBertSumProcessedData.download(local_path=PROCESSED_DATA_PATH)
    train_dataset, test_dataset = ExtSumProcessedData().splits(root=PROCESSED_DATA_PATH)
    

### Model training
To start model training, we need to create a instance of ExtractiveSummarizer.
#### Choose the transformer model.
Currently ExtractiveSummarizer support two models:
- distilbert-base-uncase, 
- bert-base-uncase

Potentionally, roberta-based model and xlnet can be supported but needs to be tested.
#### Choose the encoder algorithm.
There are four options:
- baseline: it used a smaller transformer model to replace the bert model and with transformer summarization layer
- classifier: it uses pretrained BERT and fine-tune BERT with **simple logistic classification** summarization layer
- transformer: it uses pretrained BERT and fine-tune BERT with **transformer** summarization layer
- RNN: it uses pretrained BERT and fine-tune BERT with **LSTM** summarization layer

In [29]:
# notebook parameters
# the cache data path during find tuning
CACHE_DIR = TemporaryDirectory().name

# batch size, unit is the number of tokens
BATCH_SIZE = 3000

# GPU used for training
NUM_GPUS = torch.cuda.device_count()

# Encoder name. Options are: 1. baseline, classifier, transformer, rnn.
ENCODER = "transformer"

# Learning rate
LEARNING_RATE=2e-3

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=100

# total number of steps for training
MAX_STEPS=5e2
# number of steps for warm up
WARMUP_STEPS=5e2
    
if not QUICK_RUN:
    MAX_STEPS=5e4
    WARMUP_STEPS=5e3
 

In [30]:
summarizer = ExtractiveSummarizer(MODEL_NAME, ENCODER, CACHE_DIR)

I0103 05:31:28.485541 140060135520064 file_utils.py:319] https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp8gqb629n
100%|██████████| 492/492 [00:00<00:00, 636323.64B/s]
I0103 05:31:28.695636 140060135520064 file_utils.py:334] copying /tmp/tmp8gqb629n to cache at /tmp/tmp_b2wqaou/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I0103 05:31:28.696582 140060135520064 file_utils.py:338] creating metadata file for /tmp/tmp_b2wqaou/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I0103 05:31:28.698467 140060135520064 file_utils.py:347] removing temp file /tmp/tmp8gqb629n
I0103 05:31:28.699253 140060135520064 configuration_utils.py:157] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert

In [32]:
summarizer.fit(
            train_dataset,
            num_gpus=NUM_GPUS,
            batch_size=BATCH_SIZE,
            gradient_accumulation_steps=2,
            max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
            clip_grad_norm=False,
        )

/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


loss: 10.797444, time: 48.076998, number of examples in current step: 5, step 100 out of total 1000
loss: 10.033221, time: 36.946141, number of examples in current step: 5, step 200 out of total 1000
loss: 9.813506, time: 37.003569, number of examples in current step: 5, step 300 out of total 1000
loss: 9.743949, time: 36.684293, number of examples in current step: 5, step 400 out of total 1000
loss: 9.624907, time: 36.727618, number of examples in current step: 5, step 500 out of total 1000
loss: 9.359334, time: 36.721974, number of examples in current step: 5, step 600 out of total 1000
loss: 8.998051, time: 36.738466, number of examples in current step: 6, step 700 out of total 1000
loss: 8.392073, time: 36.622983, number of examples in current step: 5, step 800 out of total 1000
loss: 7.814545, time: 36.219987, number of examples in current step: 5, step 900 out of total 1000
loss: 6.793788, time: 36.647171, number of examples in current step: 5, step 1000 out of total 1000


In [33]:
summarizer.save_model(
    os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ),
    )
)

I0103 05:38:19.590131 140060135520064 extractive_summarization.py:729] Saving model checkpoint to /tmp/tmp_b2wqaou/fine_tuned/extsum_modelname_distilbert-base-uncased_usepreprocessFalse_steps_1000.0.pt


In [22]:
# for loading a previous saved model
# import torch
# summarizer.model.load_state_dict(torch.load("bertsum_cnndm_ext.pt", map_location="cpu")) 

/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'transformers.modeling_distilbert.DistilBertModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluating text summarization.

In [34]:
target = [i['tgt_txt'] for i in test_dataset]

In [35]:
len(target)

1000

In [36]:
test_dataset[0].keys()

dict_keys(['src', 'labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])

In [37]:
%%time
prediction = summarizer.predict(test_dataset, num_gpus=NUM_GPUS, batch_size=128)

Evaluating: 100%|██████████| 8/8 [00:03<00:00,  2.71it/s]

CPU times: user 21.6 s, sys: 19.4 s, total: 41.1 s
Wall time: 3.14 s


In [38]:
len(prediction)

1000

In [39]:
RESULT_DIR = TemporaryDirectory().name

In [40]:
rouge_score = get_rouge(prediction, target, RESULT_DIR)

1000
1000


2020-01-03 05:38:27,106 [MainThread  ] [INFO ]  Writing summaries.
I0103 05:38:27.106614 140060135520064 pyrouge.py:525] Writing summaries.
2020-01-03 05:38:27,108 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmp46h_l87z/tmpemn2roi7/system and model files to /tmp/tmp46h_l87z/tmpemn2roi7/model.
I0103 05:38:27.108142 140060135520064 pyrouge.py:518] Processing summaries. Saving system files to /tmp/tmp46h_l87z/tmpemn2roi7/system and model files to /tmp/tmp46h_l87z/tmpemn2roi7/model.
2020-01-03 05:38:27,109 [MainThread  ] [INFO ]  Processing files in /tmp/tmp46h_l87z/rouge-tmp-2020-01-03-05-38-26/candidate/.
I0103 05:38:27.109484 140060135520064 pyrouge.py:43] Processing files in /tmp/tmp46h_l87z/rouge-tmp-2020-01-03-05-38-26/candidate/.
2020-01-03 05:38:27,208 [MainThread  ] [INFO ]  Saved processed files to /tmp/tmp46h_l87z/tmpemn2roi7/system.
I0103 05:38:27.208842 140060135520064 pyrouge.py:53] Saved processed files to /tmp/tmp46h_l87z/tmpemn2roi7/system.
20

---------------------------------------------
1 ROUGE-1 Average_R: 0.43558 (95%-conf.int. 0.42501 - 0.44611)
1 ROUGE-1 Average_P: 0.21518 (95%-conf.int. 0.20966 - 0.22085)
1 ROUGE-1 Average_F: 0.27915 (95%-conf.int. 0.27249 - 0.28550)
---------------------------------------------
1 ROUGE-2 Average_R: 0.15229 (95%-conf.int. 0.14301 - 0.16166)
1 ROUGE-2 Average_P: 0.07242 (95%-conf.int. 0.06794 - 0.07666)
1 ROUGE-2 Average_F: 0.09525 (95%-conf.int. 0.08952 - 0.10088)
---------------------------------------------
1 ROUGE-L Average_R: 0.38633 (95%-conf.int. 0.37639 - 0.39645)
1 ROUGE-L Average_P: 0.19072 (95%-conf.int. 0.18537 - 0.19600)
1 ROUGE-L Average_F: 0.24743 (95%-conf.int. 0.24093 - 0.25324)



In [41]:
test_dataset[0]['tgt_txt']

'marseille prosecutor says `` so far no videos were used in the crash investigation `` despite media reports .<q>journalists at bild and paris match are `` very confident `` the video clip is real , an editor says .<q>andreas lubitz had informed his lufthansa training school of an episode of severe depression , airline says .<q>'

In [42]:
prediction[0]

'marseille , france ( cnn ) the french prosecutor leading an investigation into the crash of germanwings flight 9525 insisted wednesday that he was not aware of any video footage from on board the plane .<q>marseille prosecutor brice robin told cnn that `` so far no videos were used in the crash investigation . ``<q>the two publications described the supposed video , but did not post it on their websites .'

In [43]:
test_dataset[0]['src_txt']

['marseille , france ( cnn ) the french prosecutor leading an investigation into the crash of germanwings flight 9525 insisted wednesday that he was not aware of any video footage from on board the plane .',
 'marseille prosecutor brice robin told cnn that `` so far no videos were used in the crash investigation . ``',
 'he added , `` a person who has such a video needs to immediately give it to the investigators . ``',
 "robin 's comments follow claims by two magazines , german daily bild and french paris match , of a cell phone video showing the harrowing final seconds from on board germanwings flight 9525 as it crashed into the french alps .",
 'all 150 on board were killed .',
 'paris match and bild reported that the video was recovered from a phone at the wreckage site .',
 'the two publications described the supposed video , but did not post it on their websites .',
 'the publications said that they watched the video , which was found by a source close to the investigation . ``',

In [44]:
# for testing
sb.glue("rouge_2_f_score", rouge_score['rouge_2_f_score'])

## Clean up temporary folders

In [ ]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR, ignore_errors=True)
if os.path.exists(RESULT_DIR):
    shutil.rmtree(RESULT_DIR, ignore_errors=True)
if USE_PREPROCSSED_DATA:
    if os.path.exists(PROCESSED_DATA_PATH):
        shutil.rmtree(PROCESSED_DATA_PATH, ignore_errors=True)